<a href="https://colab.research.google.com/github/xchen224/skorch/blob/main/cloud_datapro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####
def tropomi_l1brad_to_refl(l1bdata,
                     bandname,
                     region=[-180,180,-90,90],
                     irr_filename=None,
                     channel=None):
  """
  Convolve tropomi L1B radiance to reflectance 
  """

  mask3d, mask = tropomi_l1b_prescreen(l1bdata['radiance'],
                                         l1bdata['latitude'],
                                         l1bdata['longitude'], region,
                                         l1bdata['measurement_quality'],
                                         l1bdata['ground_pixel_quality'],
                                         l1bdata['spectral_channel_quality'])

  npixel = l1bdata['radiance'].shape[2]
  nchannel = l1bdata['radiance'].shape[3]
  nscan = l1bdata['radiance'].shape[1]

  print(npixel, nchannel, nscan)
  #print(mask[1265:1267,150], type(mask[1265:1267,150]))
  mask0 = np.zeros(mask.shape)
  mask0[:] = False

  # define output data
  theta0 = l1bdata['solar_zenith_angle'][0, :, :]
  theta0.mask = mask
  theta = l1bdata['viewing_zenith_angle'][0, :, :]
  theta.mask = mask
  phi0 = l1bdata['solar_azimuth_angle'][0, :, :]
  phi0.mask = mask
  phi = l1bdata['viewing_azimuth_angle'][0, :, :]
  phi.mask = mask
  lat = l1bdata['latitude'][0, :, :]
  lat.mask = mask0
  lon = l1bdata['longitude'][0, :, :]
  lon.mask = mask0
    #print('lat', lat[:50,:100], 'lon', lon[:50,:100])
    #sys.exit()
  time0 = np.reshape(np.tile(l1bdata['time'][0, :], npixel), (npixel, nscan))
  time = time0.T
    time.mask = mask0
    #print('time', time.shape, time)
    #sys.exit()

    radiance = l1bdata['radiance'][0, :, :, 0]
    refl = l1bdata['radiance'][0, :, :, 0]
    radiance.mask = mask
    refl.mask = mask

    #print(type(lat),lat[25,:])
    if use_trop_irr:
        if irr_filename is None:
            print('Error: TROPOMI solar irradiance filename is not defined!')
            sys.exit()
        else:
            irr, invalid0 = read_tropomi_irr(irr_filename, channel)
            invalid = np.ma.filled(invalid0, fill_value=True)
            print(
                'invalid array:',
                invalid.shape,
                type(invalid),
                invalid,
            )

    else:
        invalid = np.full((npixel, nchannel), False)
        print('invalid array:', invalid)

    # read tropomi filter weights
    for ip in range(npixel):
        trop = read_tropomi_filter(ip, bandname)
        #print('ip',ip)

        for ins in range(nscan):
            W0, rad, E0 = 0.0, 0.0, 0.0
            #print('ins',ins)

            if ~mask[ins, ip]:

                pifactor = np.pi / np.cos(theta0[ins, ip] * np.pi / 180.)
                #print('pifac',pifactor)

                for ic in range(nchannel - 1):

                    #print(ins, ip, ic, invalid[ip,ic])
                    if ~invalid[ip, ic]:

                        #if ~mask3d[ins,ip,ic] and ~mask3d[ins,ip,ic+1]:
                        #if ic < nchannel - 1:
                        deltaw = trop['lamda'][ic + 1] - trop['lamda'][ic]
                        #else:
                        #deltaw = trop['lamda'][ic] - trop['lamda'][ic-1]

                        #print(deltaw)
                        # transfer units of radiance (E = nhf = nh(c/lamda))
                        erad = l1bdata['radiance'][
                            0, ins, ip,
                            ic] * 6.022E23 * 6.626E-34 * 3.0E8 / trop['lamda'][
                                ic] * 1.0E9
                        W0 += trop['f'][ic] * deltaw
                        rad += trop['f'][ic] * erad * deltaw
                        if use_trop_irr:
                            eirr = irr['irradiance'][
                                ip, ic] * 6.022E23 * 6.626E-34 * 3.0E8 / trop[
                                    'lamda'][ic] * 1.0E9
                            E0 += trop['f'][ic] * eirr * deltaw
                        else:
                            E0 += trop['f'][ic] * trop['e0'][ic] * deltaw

                        #print('erad', erad, W0, rad, E0)

                radiance[ins, ip] = rad / W0
                E0 = E0 / W0
                refl[ins, ip] = rad / W0 / E0 * pifactor
                #print (radiance[ins,ip],refl[ins,ip])

    return dict(rad=radiance,
                refl=refl,
                lat=lat,
                lon=lon,
                theta0=theta0,
                theta=theta,
                phi0=phi0,
                phi=phi,
                time=time)
